In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from seaborn import set_style
import kagglehub
set_style("whitegrid")

In [4]:
path = kagglehub.dataset_download("ehallmar/daily-historical-stock-prices-1970-2018")

print("Path to dataset files:", path)

Path to dataset files: /home/hava/.cache/kagglehub/datasets/ehallmar/daily-historical-stock-prices-1970-2018/versions/1


In [5]:
prices_file = path + '/historical_stock_prices.csv'
labels_file = path + '/historical_stocks.csv'
df_prices = pd.read_csv(prices_file)
df_labels = pd.read_csv(labels_file)
df_prices

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14
...,...,...,...,...,...,...,...,...
20973884,NZF,14.60,14.59,14.590000,14.58,14.62,137500,2018-08-20
20973885,NZF,14.60,14.58,14.580000,14.57,14.61,151200,2018-08-21
20973886,NZF,14.58,14.59,14.590000,14.57,14.63,185400,2018-08-22
20973887,NZF,14.60,14.57,14.570000,14.57,14.64,135600,2018-08-23


In [13]:
sector = df_labels[df_labels['sector']=='HEALTH CARE'][['ticker','name']]
sector['ticker']=sector['ticker'].astype(str)
print(sector['ticker'])
df_sector = df_prices[df_prices['ticker'].isin(sector['ticker'])].reset_index()
df_sector

14       ABEO
15      ABEOW
17       ABMD
18       ABLX
21       ACIU
        ...  
6259      VAR
6349      WCG
6450      ZBH
6456      ZTS
6459     ZYME
Name: ticker, Length: 784, dtype: object


,index,ticker,open,close,adj_close,low,high,volume,date
0,730,GHDX,12.000000,11.750000,11.750000,11.75,12.250000,2628300,2005-09-29
1,736,GHDX,12.000000,11.550000,11.550000,11.55,12.000000,219900,2005-09-30
2,742,GHDX,11.980000,11.690000,11.690000,11.65,11.980000,137400,2005-10-03
3,748,GHDX,11.720000,11.550000,11.550000,11.45,11.800000,228000,2005-10-04
4,756,GHDX,11.700000,10.350000,10.350000,9.87,11.700000,1010000,2005-10-05
...,...,...,...,...,...,...,...,...,...
2093688,20960696,AKRX,19.100000,19.480000,19.480000,18.98,19.549999,2093400,2018-08-20
2093689,20960697,AKRX,19.400000,17.920000,17.920000,17.84,19.420000,2682000,2018-08-21
2093690,20960698,AKRX,17.820000,18.110001,18.110001,17.60,18.400000,1566900,2018-08-22
2093691,20960699,AKRX,18.360001,14.930000,14.930000,13.74,18.440001,12820400,2018-08-23


In [14]:
df_sector['date'] = pd.to_datetime(df_sector['date'])
df_sector['ticker'] = (df_sector['ticker']).astype(str)
dst_dates_df = pd.read_csv('DaylightSavingsTimeChangeDates_1971-2024.csv')
dst_dates_spring = pd.to_datetime(dst_dates_df.Date+" "+dst_dates_df.Year.astype(str))
dst_dates_fall = pd.to_datetime(dst_dates_df['Date.1']+" "+dst_dates_df.Year.astype(str))
dst_dates = pd.concat([dst_dates_fall,dst_dates_spring])
dst_mon = dst_dates+pd.DateOffset(1)
dst_fri = dst_dates+pd.DateOffset(-2)
dst_fall_mon = dst_dates_fall+pd.DateOffset(1)
dst_fall_fri = dst_dates_fall+pd.DateOffset(-2)
dst_spring_mon = dst_dates_spring+pd.DateOffset(1)
dst_spring_fri = dst_dates_spring+pd.DateOffset(-2)

In [15]:
sector_dst_mon_fall = df_sector[df_sector.date.isin(dst_fall_mon)]
sector_dst_mon_fall = sector_dst_mon_fall.assign(Season='Fall')
sector_dst_mon_spring = df_sector[df_sector.date.isin(dst_spring_mon)]
sector_dst_mon_spring = sector_dst_mon_spring.assign(Season='Spring')
sector_dst_mon = pd.concat([sector_dst_mon_fall,sector_dst_mon_spring])
sector_dst_fri = df_sector[df_sector.date.isin(dst_fri)]

for tkr in sector['ticker'].astype(str):
    mon_tkr = sector_dst_mon[sector_dst_mon.ticker.astype(str)==tkr]
    fri_tkr = sector_dst_fri[sector_dst_fri.ticker.astype(str)==tkr]
    sector_dst_mon = sector_dst_mon.drop(mon_tkr[mon_tkr.date.isin(fri_tkr.date+pd.DateOffset(3))==False].index)
    sector_dst_fri = sector_dst_fri.drop(fri_tkr[fri_tkr.date.isin(mon_tkr.date+pd.DateOffset(-3))==False].index)
    #finance_dst_mon = finance_dst_mon.drop(finance_dst_mon[(finance_dst_mon.ticker.astype(str)==tkr  finance_dst_mon['date'].isin(finance_dst_fri[finance_dst_fri.ticker.astype(str)==tkr].date+pd.DateOffset(3))==False)].index)
    #finance_dst_fri = finance_dst_fri.drop(finance_dst_fri[(finance_dst_fri['ticker'==tkr] & finance_dst_fri['date'].isin(finance_dst_mon['ticker'==tkr].date+pd.DateOffset(-3))==False)].index)
print(len(sector_dst_mon))
print(len(sector_dst_fri))

15729
15729


In [16]:
sector_dst_fri = sector_dst_fri.reset_index()
sector_dst_mon = sector_dst_mon.reset_index()

In [17]:
sector_dst = pd.DataFrame()
sector_dst['ticker'] = sector_dst_mon['ticker']
sector_dst['mon_date'] = sector_dst_mon['date']
sector_dst['mon_open'] = sector_dst_mon['open']
sector_dst['mon_close'] = sector_dst_mon['close']
sector_dst['mon_adj_close'] = sector_dst_mon['adj_close']
sector_dst['mon_low'] = sector_dst_mon['low']
sector_dst['mon_high'] = sector_dst_mon['high']
sector_dst['mon_volume'] = sector_dst_mon['volume']
sector_dst['fri_date'] = sector_dst_fri['date']
sector_dst['fri_open'] = sector_dst_fri['open']
sector_dst['fri_close'] = sector_dst_fri['close']
sector_dst['fri_adj_close'] = sector_dst_fri['adj_close']
sector_dst['fri_low'] = sector_dst_fri['low']
sector_dst['fri_high'] = sector_dst_fri['high']
sector_dst['fri_volume'] = sector_dst_fri['volume']
sector_dst['Season'] = sector_dst_mon['Season']
sector_dst

,ticker,mon_date,mon_open,mon_close,mon_adj_close,mon_low,mon_high,mon_volume,fri_date,fri_open,fri_close,fri_adj_close,fri_low,fri_high,fri_volume,Season
0,GHDX,2005-10-31,9.650000,9.530000,9.530000,9.400000,9.800000,42300,2005-10-28,9.150000,9.430000,9.430000,9.100000,9.490000,27000,Fall
1,GHDX,2006-10-30,16.129999,16.790001,16.790001,15.830000,17.090000,87800,2006-03-31,10.240000,10.370000,10.370000,9.950000,10.540000,1356900,Fall
2,GHDX,2007-11-05,23.910000,23.690001,23.690001,23.490000,24.250000,120000,2006-10-27,16.760000,16.129999,16.129999,16.020000,16.910000,66700,Fall
3,GHDX,2008-11-03,18.430000,18.910000,18.910000,18.430000,19.250000,132300,2007-03-09,18.660000,18.540001,18.540001,18.260000,18.730000,326500,Fall
4,GHDX,2009-11-02,18.590000,18.879999,18.879999,18.309999,18.910000,161900,2007-11-02,24.440001,24.110001,24.110001,23.840000,24.660000,137400,Fall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15724,AKRX,2014-03-10,22.209999,22.610001,22.610001,22.139999,22.770000,870900,2016-03-11,24.340000,24.650000,24.650000,24.049999,24.740000,1195400,Spring
15725,AKRX,2015-03-09,49.830002,51.840000,51.840000,49.740002,52.020000,1580800,2016-11-04,20.270000,19.680000,19.680000,19.280001,21.080000,3827300,Spring
15726,AKRX,2016-03-14,24.740000,23.709999,23.709999,23.680000,24.889999,1300000,2017-03-10,22.969999,22.660000,22.660000,22.309999,22.990000,951100,Spring
15727,AKRX,2017-03-13,22.719999,22.500000,22.500000,22.330000,22.860001,844600,2017-11-03,33.200001,33.150002,33.150002,33.090000,33.270000,1693300,Spring


In [18]:
sector_mon = df_sector[df_sector.date.dt.day_name()=='Monday']
sector_mon = sector_mon.drop(sector_mon[sector_mon.date.isin(dst_mon)].index)
sector_fri = df_sector[df_sector.date.dt.day_name()=='Friday']
sector_fri = sector_fri.drop(sector_fri[sector_fri.date.isin(dst_fri)].index)
for tkr in sector['ticker'].astype(str):
    mon_tkr = sector_mon[sector_mon.ticker.astype(str)==tkr]
    fri_tkr = sector_fri[sector_fri.ticker.astype(str)==tkr]
    sector_mon = sector_mon.drop(mon_tkr[mon_tkr.date.isin(fri_tkr.date+pd.DateOffset(3))==False].index)
    sector_fri = sector_fri.drop(fri_tkr[fri_tkr.date.isin(mon_tkr.date+pd.DateOffset(-3))==False].index)
print(len(sector_mon))
print(len(sector_fri))

356233
356233


In [19]:
sector_mon = sector_mon.reset_index()
sector_fri = sector_fri.reset_index()

In [20]:
sector_wkd = pd.DataFrame()
sector_wkd['ticker'] = sector_mon['ticker']
sector_wkd['mon_date'] = sector_mon['date']
sector_wkd['mon_open'] = sector_mon['open']
sector_wkd['mon_close'] = sector_mon['close']
sector_wkd['mon_adj_close'] = sector_mon['adj_close']
sector_wkd['mon_low'] = sector_mon['low']
sector_wkd['mon_high'] = sector_mon['high']
sector_wkd['mon_volume'] = sector_mon['volume']
sector_wkd['fri_date'] = sector_fri['date']
sector_wkd['fri_open'] = sector_fri['open']
sector_wkd['fri_close'] = sector_fri['close']
sector_wkd['fri_adj_close'] = sector_fri['adj_close']
sector_wkd['fri_low'] = sector_fri['low']
sector_wkd['fri_high'] = sector_fri['high']
sector_wkd['fri_volume'] = sector_fri['volume']
sector_wkd

,ticker,mon_date,mon_open,mon_close,mon_adj_close,mon_low,mon_high,mon_volume,fri_date,fri_open,fri_close,fri_adj_close,fri_low,fri_high,fri_volume
0,GHDX,2005-10-03,11.980000,11.690000,11.690000,11.650000,11.980000,137400,2005-09-30,12.000000,11.550000,11.550000,11.550000,12.000000,219900
1,GHDX,2005-10-10,10.850000,10.510000,10.510000,10.510000,10.850000,6100,2005-10-07,10.600000,10.500000,10.500000,10.380000,10.650000,220100
2,GHDX,2005-10-17,9.950000,9.920000,9.920000,9.800000,9.990000,47700,2005-10-14,9.850000,9.780000,9.780000,9.690000,9.850000,83300
3,GHDX,2005-10-24,9.250000,9.190000,9.190000,9.000000,9.300000,39500,2005-10-21,8.850000,9.000000,9.000000,8.560000,9.240000,206200
4,GHDX,2005-11-07,10.200000,10.350000,10.350000,10.140000,10.400000,17400,2005-11-04,10.240000,10.170000,10.170000,9.970000,10.250000,175800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356228,AKRX,2018-07-23,17.340000,17.270000,17.270000,17.200001,17.480000,1177100,2018-07-20,17.350000,17.469999,17.469999,17.150000,17.500000,1102900
356229,AKRX,2018-07-30,18.139999,18.309999,18.309999,18.000000,18.520000,918900,2018-07-27,17.600000,18.230000,18.230000,17.590000,18.290001,1054700
356230,AKRX,2018-08-06,18.250000,19.030001,19.030001,18.250000,19.049999,1669800,2018-08-03,17.799999,18.600000,18.600000,17.799999,18.639999,1276400
356231,AKRX,2018-08-13,18.719999,18.719999,18.719999,18.420000,18.950001,1145200,2018-08-10,18.790001,18.590000,18.590000,18.450001,19.020000,1081200


In [21]:
sector_dst.to_csv('DST_Health_Stocks.csv')
sector_wkd.to_csv('Weekends_Health_Stocks.csv')